In [1]:
import sys
sys.path.append('../..')

In [2]:
from pathlib import Path

data_root = Path('/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372')

## Clean meshes

In [3]:
from glob import glob

mesh_files = sorted(glob(str(data_root / 'meshes' / '*.obj')))
mesh_files

['/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000001.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000002.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000003.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000004.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000005.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000006.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000007.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0372.000008.obj',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/meshes/S6_nude_jump120_0309_0

In [ ]:
import os
from tqdm import tqdm
import pyvista as pv
pv.set_jupyter_backend('html')

os.makedirs(data_root / 'clean', exist_ok=True)

for mesh_file in tqdm(mesh_files, desc='Processing meshes'):
    mesh_clean = pv.read(mesh_file).clean().split_bodies()[0].extract_surface().triangulate()
    mesh_clean.save(str(data_root / 'clean' / Path(mesh_file).stem) + '.ply')

Processing meshes:   8%|▊         | 5/64 [00:00<00:07,  7.83it/s]

Processing meshes: 100%|██████████| 64/64 [00:08<00:00,  7.73it/s]


## Simplify meshes

In [5]:
import pymeshlab as ml

def mesh_simplify(input_mesh_path, output_mesh_path, target_vertex_count):
    ms = ml.MeshSet()
    ms.load_new_mesh(input_mesh_path)
    m = ms.current_mesh()

    # clean-up in case of input has duplicate/unreferenced vertices
    ms.meshing_remove_duplicate_vertices()
    ms.meshing_remove_unreferenced_vertices()

    # initial estimation number of faces, designed to be superior than the teorical value
    numFaces = 100 + ms.current_mesh().face_number() - (ms.current_mesh().vertex_number() - target_vertex_count)

    # iteratively simplify the mesh
    while (ms.current_mesh().vertex_number() > target_vertex_count):
        ms.meshing_decimation_quadric_edge_collapse(targetfacenum=numFaces, preservenormal=True)
        numFaces = numFaces - (ms.current_mesh().vertex_number() - target_vertex_count)

    m = ms.current_mesh()
    ms.save_current_mesh(output_mesh_path)#, save_textures=False)

In [6]:
mesh_clean_files = sorted(glob(str(data_root / 'clean' / '*.ply')))
mesh_clean_files

['/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000001.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000002.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000003.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000004.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000005.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000006.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000007.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.000008.ply',
 '/home/knpob/Documents/Hinton/data/3dMD/S6_nude_jump120_0309_0372/clean/S6_nude_jump120_0309_0372.00000

In [7]:
import os
from tqdm import tqdm

os.makedirs(data_root / 'ply', exist_ok=True)

for mesh_file in tqdm(mesh_clean_files, desc='Processing meshes'):
    mesh_simplify(
        input_mesh_path=mesh_file,
        output_mesh_path=str(data_root / 'ply' / Path(mesh_file).stem) + '.ply',
        target_vertex_count=5000,
    )

Processing meshes: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]


In [11]:
pv.read(str(data_root / 'ply' / Path(mesh_file).stem) + '.ply').plot()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [9]:
import shutil

# remove the cleaned meshes directory
shutil.rmtree(data_root / 'clean')